# 4.2 Feature Engineering & 연관규칙 데이터 생성

- 각 **연관규칙** 카테고리별, 결측치 처리 방법별 모델링에 사용할 데이터 생성
- 이 데이터를 사용하여 5_LG_고객데이터_결측치_방법론선택.ipynb 파일을 **수정하여** 연관규칙 카테고리에 맞는 결측치 처리 방법 선택하여야함

In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime

pd.set_option('display.max_columns', None)


In [23]:
df = pd.read_csv('../data/결측치_output/결측치_ArbitaryImputer.csv', index_col = 0)
# df = pd.read_csv('../data/결측치_KNNImputer_distance10_final.csv', index_col = 0)


/Users/hyojinlee/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
df['최근구매채널'].value_counts()

4          374520
3           46293
2           44813
1           34231
4           24832
3            3029
2            2607
1            2296
Missing         4
Name: 최근구매채널, dtype: int64

In [25]:
df.columns

Index(['고객ID', '매출일자(배송완료일자)', '타겟제품명', '금액', '멤버십고객여부', '케어솔루션품목수', '케어십가입여부',
       '케어십품목수', '최근3년구매금액', '최근구매채널', 'Only&Best품목수', '건강관리가전품목수', '연령대',
       '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도', '보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부',
       '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부', '거주광역명', '거주시군구명', '년도', '월'],
      dtype='object')

## 2023년에 구매한 이력은 제거

In [26]:
df.loc[df['년도'] == 2023, 'label_2023'] = 1
df.loc[df['년도'] != 2023, 'label_2023'] = 0

In [27]:
df_isbuy_2023 = df[['고객ID', 'label_2023']]
df_isbuy_2023 = df_isbuy_2023[df_isbuy_2023['label_2023'] == 1]
df_isbuy_2023.drop_duplicates("고객ID", inplace = True)
df_isbuy_2023.rename({'label_2023':'label_y'}, axis = 1, inplace = True)
df_isbuy_2023

,고객ID,label_y
6250,40445,1.0
9260,24253,1.0
12405,63417,1.0
16373,22718,1.0
18475,20032,1.0
...,...,...
532602,91470,1.0
532609,92672,1.0
532612,98487,1.0
532616,102635,1.0


In [28]:
# 꼭 돌려야함
df = df.loc[df['label_2023'] != 1]

In [29]:
df_crm = pd.read_csv('../data/master_crm.csv', encoding='cp949')

## 제품군 별로 나누기 - 연관규칙

그룹1: TV, 에어컨, 냉장고, 세탁기
그룹2: 건조기, 스타일러, 공기청정기, 청소기
그룹3: 김치냉장고, 식기세척기, 전기레인지, 정수기
그룹4: 미니워시, PC, 휴대폰, 뷰티기기
그룹5: 모니터, 안마의자


In [30]:
df['타겟제품명'] = df['타겟제품명'].replace({
    '시그니처냉장고': '냉장고',
    '시그니처세탁기': '세탁기',
    '시그니처공청기': '공기청정기',
    '시그니처에어컨': '에어컨',
    '시그니처TV': 'TV'
})

In [31]:
#### 수정 필요
def split_category(x):
    if x in ['TV', '에어컨', '냉장고', '세탁기']:
        new_c = '그룹1'
    elif x in ['건조기', '스타일러', '공기청정기', '청소기']:
        new_c = '그룹2'
    elif x in ['김치냉장고', '식기세척기', '전기레인지', '정수기']:
        new_c = '그룹3'
    elif x in ['미니워시', 'PC', '휴대폰', '뷰티기기']:
        new_c = '그룹4'
    elif x in ['모니터', '안마의자']:
        new_c = '그룹5'
    elif x in ['로봇청소기', '얼음정수기냉장고', 'AV', '워시타워']:
        new_c = '그룹6'
    else:
        new_c = '기타'
    return new_c

In [32]:
df['제품카테고리'] = df['타겟제품명'].apply(lambda x: split_category(x))

## 가공 변수 추가

- df_crm에 가공변수를 추가 
- 마지막에 df_crm과 제품별로 나눠진 데이터를 merge 필요

### 총구매빈도

In [33]:
# df.loc[df['년도'] == 2023, 'label_2023'] = 1
# df.loc[df['년도'] != 2023, 'label_2023'] = 0

In [34]:
df_crm = df_crm.merge(df.groupby(['고객ID']).count()['label_2023'], on = "고객ID")
df_crm = df_crm.rename(columns = {"label_2023" : "총구매빈도"})
df_crm.head(3)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3
2,3,NaN,1.0,0품목,0,0품목,0원,하이마트,1품목,2품목,NaN,아파트,5~6억,30평대,미사용,NaN,NaN,0,0,0,NaN,경기,화성시,새솔동,0,5


### 최대구매가격대

In [35]:
# df['금액_중간값'] = df['금액'].replace(to_replace={"0만원 이상 100만원 이하":50 , "100만원 이상 200만원 이하":150, "200만원 이상 300만원 이하":250, "300만원 이상 400만원 이하":350, "400만원 이상 500만원 이하":450, "500만원 이상 600만원 이하":550, "600만원 이상 700만원 이하":650, "700만원 이상 800만원 이하":750, "800만원 이상 900만원 이하":850, "900만원 이상 1000만원 이하":950, "1000만원 이상":1000 })

In [36]:
# 결측치_ArbitaryImputer.csv 인 경우에 셀 실행

df['금액_중간값'] = df['금액'].replace(to_replace={"Missing": 0, "0만원 이상 100만원 이하":50 , "100만원 이상 200만원 이하":150, "200만원 이상 300만원 이하":250, "300만원 이상 400만원 이하":350, "400만원 이상 500만원 이하":450, "500만원 이상 600만원 이하":550, "600만원 이상 700만원 이하":650, "700만원 이상 800만원 이하":750, "800만원 이상 900만원 이하":850, "900만원 이상 1000만원 이하":950, "1000만원 이상":1000 })



In [37]:
# 고객별로 그룹화하 후 각 그룹에서 최대값 리턴
max_purchase_by_customer = df.groupby('고객ID')['금액_중간값'].max()

In [38]:
df_crm['최대구매가격대'] = max_purchase_by_customer.reset_index(drop = True)
df_crm.head(2)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도,최대구매가격대
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3,250
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3,150


### 가장 최근 구매 이후 경과일

In [136]:

# 날짜 형식으로 변환
df['매출일자(배송완료일자)'] = pd.to_datetime(df['매출일자(배송완료일자)'])

# 현재 날짜 계산
current_date = datetime.now()

# 각 고객 별로 최근 매출일자(배송완료일자)를 찾고 일수 차이 계산
customer_last_purchase_dates = df.groupby('고객ID')['매출일자(배송완료일자)'].max().reset_index()
customer_last_purchase_dates['최근매출일로부터경과일'] = (current_date - customer_last_purchase_dates['매출일자(배송완료일자)']).dt.days

customer_last_purchase_dates

,고객ID,매출일자(배송완료일자),최근매출일로부터경과일
0,1,2022-01-11,703
1,2,2021-12-29,716
2,3,2020-08-17,1215
3,4,2021-06-05,923
4,5,2020-10-08,1163
...,...,...,...
102718,104347,2023-03-03,287
102719,104348,2023-07-15,153
102720,104349,2023-05-01,228
102721,104350,2023-03-11,279


In [137]:
df_crm = df_crm.merge(customer_last_purchase_dates[['고객ID', '매출일자(배송완료일자)', '최근매출일로부터경과일']], on = "고객ID")

### 가장 선호하는 구매 제품

In [39]:
# 각 고객ID 별로 제품 카테고리의 구매 횟수를 계산
customer_purchase_counts = df.groupby(['고객ID', '제품카테고리']).size().reset_index(name='구매횟수')

# 각 고객ID 별로 가장 높은 구매 횟수를 찾아 해당하는 제품 카테고리를 반환
max_purchase_categories = customer_purchase_counts.groupby('고객ID').apply(lambda x: x[x['구매횟수'] == x['구매횟수'].max()]['제품카테고리'].tolist())

max_purchase_categories

고객ID
1              [그룹3]
2              [그룹3]
3              [그룹1]
4              [그룹2]
5         [그룹2, 그룹4]
             ...    
104347    [그룹1, 그룹5]
104348    [그룹2, 그룹3]
104349    [그룹1, 그룹3]
104350         [그룹1]
104351    [그룹1, 그룹2]
Length: 102691, dtype: object

In [40]:
df_crm['주구매제품카테고리'] = max_purchase_categories.reset_index(drop = True)
df_crm.head(2)

,고객ID,멤버십고객여부,케어솔루션고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,거주행정동명,label,총구매빈도,최대구매가격대,주구매제품카테고리
0,1,NaN,0.0,0품목,1,2품목,5KK이상10KK미만,홈플러스,2품목,2품목,NaN,아파트,1억↓,20평대,미사용,NaN,NaN,0,0,0,NaN,경기,안산시 상록구,일동,0,3,250,[그룹3]
1,2,NaN,0.0,1품목,0,0품목,1KK미만,B2B,2품목,2품목,NaN,아파트,3~4억,40평대,미사용,NaN,NaN,0,0,0,NaN,경기,오산시,대원동,0,3,150,[그룹3]


# 방법론 1
- Idea : 각 제품군 별로 특징이 다를 것이므로, 제품군 별로 구매가능성을 예측하는 데에 더 정확할 것
- 따라서, 제품 카테고리 별로 데이터프레임을 구분한 후, 각각 제품군 별로 모델링을 하여, 모델을 여러 개 생성
- 이때, Y_target은 각 제품군 구매여부(1, 0)가 됨.
    - 2023년의 구매이력과 매칭하여, 어떤 제품군 카테고리를 구매했는지 확인할 수 있으므로, 각 제품군 별 구매여부 추적이 가능함

기본컬럼 (23) : '고객ID', '멤버십고객여부', '케어솔루션품목수', '케어십가입여부',
       '케어십품목수', '최근3년구매금액', '최근구매채널', 'Only&Best품목수', '건강관리가전품목수', '연령대',
       '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도', '보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부',
       '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부', '거주광역명', '거주시군구명', 'label_y'
       
추가 (4의배수) : '년도', '월', '타겟제품명', '금액'

가공변수 (4) : 총구매빈도, 최대구매가격대, 최근매출일로부터경과일, 주구매제품카테고리

모델링에서 제거 : '년도', '최근매출일로부터경과일', '고객ID'

## 제품별로 데이터프레임 생성

### 모든 데이터

In [41]:
df = df.merge(df_isbuy_2023, on = '고객ID', how = 'left')
df.loc[df['label_y'].isnull(), 'label_y'] = 0

### 그룹1

In [45]:
group1_df = df.loc[df['제품카테고리'] == '그룹1']

In [46]:
group1_df['label_y'].value_counts()

0.0    267984
1.0     34526
Name: label_y, dtype: int64

### 그룹2

In [47]:
group2_df = df.loc[df['제품카테고리'] == '그룹2']

In [48]:
group2_df['label_y'].value_counts()

0.0    78073
1.0    11531
Name: label_y, dtype: int64

### 그룹3

In [49]:
group3_df = df.loc[df['제품카테고리'] == '그룹3']

group3_df['label_y'].value_counts()

0.0    50069
1.0     7664
Name: label_y, dtype: int64

### 그룹4

In [50]:
group4_df = df.loc[df['제품카테고리'] == '그룹4']


group4_df['label_y'].value_counts()

0.0    39855
1.0     5111
Name: label_y, dtype: int64

### 그룹5

In [51]:
group5_df = df.loc[df['제품카테고리'] == '그룹5']

group5_df['label_y'].value_counts()

0.0    1672
1.0     403
Name: label_y, dtype: int64

### 그룹6

In [52]:
group6_df = df.loc[df['제품카테고리'] == '그룹6']

group6_df['label_y'].value_counts()

0.0    5727
1.0     993
Name: label_y, dtype: int64

## 고객의 구매이력 데이터 통합 함수

In [54]:
def create_row_from_group(group, customer_id):

    new_row = {'고객ID': customer_id}
    for column in group.columns:
        new_row[column] = group.iloc[0][column]
    return new_row

In [55]:
def organize_purchases_per_customer(dataframe, column_list):
    
    # 단일 고객의 최대 구매 횟수 파악
    grouped_new = dataframe.groupby('고객ID')
    max_purchases_new = grouped_new.size().max()

    label_encoding_df = {column: [] for column in column_list}

    for i in range(1, max_purchases_new + 1):
        label_encoding_df[f'년도_{i}'] = []
        label_encoding_df[f'월_{i}'] = []
        label_encoding_df[f'타겟제품명_{i}'] = []
        label_encoding_df[f'금액_{i}'] = []

    for customer_id, group in tqdm(grouped_new):
        purchases = list(group[['년도', '월','타겟제품명', '금액']].itertuples(index=False, name=None))
        new_row_new = create_row_from_group(group, customer_id)

        # 새 행에 판매 날짜 및 구매 추가
        for i, (year, month, product_name, amount) in enumerate(purchases, 1):
            new_row_new[f'년도_{i}'] = year
            new_row_new[f'월_{i}'] = month
            new_row_new[f'타겟제품명_{i}'] = product_name
            new_row_new[f'금액_{i}'] = amount

        # 최대 구매수보다 적은 경우 나머지 행 채우기
        for j in range(len(purchases) + 1, max_purchases_new + 1):
            new_row_new[f'년도_{j}'] = None
            new_row_new[f'월_{j}'] = None
            new_row_new[f'타겟제품명_{j}'] = None
            new_row_new[f'금액_{j}'] = None

        for key in label_encoding_df:
            label_encoding_df[key].append(new_row_new.get(key, None))

    new_structured_df_new = pd.DataFrame(label_encoding_df)
    return new_structured_df_new


## 고객별 데이터 생성 : 고객 정보와 가공변수 포함된 df 

In [56]:
def final_df_maker(dataframe, col_list):
    df_new = organize_purchases_per_customer(dataframe, col_list)
    # 가공변수 추가
    df_new = df_new.merge(df_crm[['고객ID','총구매빈도', '최대구매가격대',
       '주구매제품카테고리']], on = '고객ID', how = 'left')
    return df_new

In [57]:
col_list = [
'고객ID','멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액', '최근구매채널',
'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대', '포인트사용빈도',
'보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '멤버십앱설치여부',
'거주광역명', '거주시군구명', 'label_y']

In [58]:
group1_df_new = final_df_maker(group1_df, col_list)
group2_df_new = final_df_maker(group2_df, col_list)
group3_df_new = final_df_maker(group3_df, col_list)
group4_df_new = final_df_maker(group4_df, col_list)
group5_df_new = final_df_maker(group5_df, col_list)
group6_df_new = final_df_maker(group6_df, col_list)

100%|██████████████████████████████████████| 5900/5900 [00:12<00:00, 462.41it/s]


In [59]:
# group1_df_new.to_csv('../data/association_rule/group1_df_값생성_2023제거.csv', encoding='utf-8-sig')
# group2_df_new.to_csv('../data/association_rule/group2_df_값생성_2023제거.csv', encoding='utf-8-sig')
# group3_df_new.to_csv('../data/association_rule/group3_df_값생성_2023제거.csv', encoding='utf-8-sig')
# group4_df_new.to_csv('../data/association_rule/group4_df_값생성_2023제거.csv', encoding='utf-8-sig')
# group5_df_new.to_csv('../data/association_rule/group5_df_값생성_2023제거.csv', encoding='utf-8-sig')
# group6_df_new.to_csv('../data/association_rule/group6_df_값생성_2023제거.csv', encoding='utf-8-sig')